In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
import urllib
import os
import csv
import cv2
import time
from PIL import Image

In [2]:
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

In [3]:
from keras.models import load_model
from keras_retinanet import models

# Load model

In [5]:
# adjust this to point to your downloaded/trained model
# models can be downloaded here: https://github.com/fizyr/keras-retinanet/releases
model_path = os.path.join('keras-retinanet', 'snapshots', sorted(os.listdir('keras-retinanet/snapshots'), reverse=True)[0])

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')

# if the model is not converted to an inference model, use the line below
# see: https://github.com/fizyr/keras-retinanet#converting-a-training-model-to-inference-model
model = models.convert_model(model)

print(model.summary())

labels_to_names = pd.read_csv('classes.csv', header=None).T.loc[0].to_dict()

Model: "retinanet-bbox"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1 (Conv2D)                 (None, None, None,   9408        ['input_1[0][0]']                
                                64)                                                               
                                                                                                  
 bn_conv1 (BatchNormalization)  (None, None, None,   256         ['conv1[0][0]']                  
                                64)                                                  

                                                                                                  
 res2b (Add)                    (None, None, None,   0           ['bn2b_branch2c[0][0]',          
                                256)                              'res2a_relu[0][0]']             
                                                                                                  
 res2b_relu (Activation)        (None, None, None,   0           ['res2b[0][0]']                  
                                256)                                                              
                                                                                                  
 res2c_branch2a (Conv2D)        (None, None, None,   16384       ['res2b_relu[0][0]']             
                                64)                                                               
                                                                                                  
 bn2c_bran

 ion)                           128)                                                              
                                                                                                  
 res3b_branch2a_relu (Activatio  (None, None, None,   0          ['bn3b_branch2a[0][0]']          
 n)                             128)                                                              
                                                                                                  
 padding3b_branch2b (ZeroPaddin  (None, None, None,   0          ['res3b_branch2a_relu[0][0]']    
 g2D)                           128)                                                              
                                                                                                  
 res3b_branch2b (Conv2D)        (None, None, None,   147456      ['padding3b_branch2b[0][0]']     
                                128)                                                              
          

 res3d_branch2c (Conv2D)        (None, None, None,   65536       ['res3d_branch2b_relu[0][0]']    
                                512)                                                              
                                                                                                  
 bn3d_branch2c (BatchNormalizat  (None, None, None,   2048       ['res3d_branch2c[0][0]']         
 ion)                           512)                                                              
                                                                                                  
 res3d (Add)                    (None, None, None,   0           ['bn3d_branch2c[0][0]',          
                                512)                              'res3c_relu[0][0]']             
                                                                                                  
 res3d_relu (Activation)        (None, None, None,   0           ['res3d[0][0]']                  
          

                                                                                                  
 res4c_branch2a (Conv2D)        (None, None, None,   262144      ['res4b_relu[0][0]']             
                                256)                                                              
                                                                                                  
 bn4c_branch2a (BatchNormalizat  (None, None, None,   1024       ['res4c_branch2a[0][0]']         
 ion)                           256)                                                              
                                                                                                  
 res4c_branch2a_relu (Activatio  (None, None, None,   0          ['bn4c_branch2a[0][0]']          
 n)                             256)                                                              
                                                                                                  
 padding4c

 ion)                           256)                                                              
                                                                                                  
 res4e_branch2b_relu (Activatio  (None, None, None,   0          ['bn4e_branch2b[0][0]']          
 n)                             256)                                                              
                                                                                                  
 res4e_branch2c (Conv2D)        (None, None, None,   262144      ['res4e_branch2b_relu[0][0]']    
                                1024)                                                             
                                                                                                  
 bn4e_branch2c (BatchNormalizat  (None, None, None,   4096       ['res4e_branch2c[0][0]']         
 ion)                           1024)                                                             
          

 res5a (Add)                    (None, None, None,   0           ['bn5a_branch2c[0][0]',          
                                2048)                             'bn5a_branch1[0][0]']           
                                                                                                  
 res5a_relu (Activation)        (None, None, None,   0           ['res5a[0][0]']                  
                                2048)                                                             
                                                                                                  
 res5b_branch2a (Conv2D)        (None, None, None,   1048576     ['res5a_relu[0][0]']             
                                512)                                                              
                                                                                                  
 bn5b_branch2a (BatchNormalizat  (None, None, None,   2048       ['res5b_branch2a[0][0]']         
 ion)     

                                                                                                  
 P4_upsampled (UpsampleLike)    (None, None, None,   0           ['P4_merged[0][0]',              
                                256)                              'res3d_relu[0][0]']             
                                                                                                  
 C3_reduced (Conv2D)            (None, None, None,   131328      ['res3d_relu[0][0]']             
                                256)                                                              
                                                                                                  
 P6 (Conv2D)                    (None, None, None,   4718848     ['res5c_relu[0][0]']             
                                256)                                                              
                                                                                                  
 P3_merged

# Prediction

In [6]:
import skimage.io as io

def predict(image):
    # Preprocess the image
    image = preprocess_image(image.copy())
    image, scale = resize_image(image)
    # Generate the prediction
    boxes, scores, labels = model.predict_on_batch(
     np.expand_dims(image, axis=0)
     )
    # Scale the output
    boxes /= scale

    return boxes, scores, labels

# Camera connection

In [8]:
camera = cv2.VideoCapture(0)
camera_height = 500

while(True):
    # Take an image (frame)
    _, frame = camera.read()
    frame = cv2.flip(frame, 1)

    # Try prediction
    try:
        boxes, scores, labels = predict(frame)
    except:
        continue
    draw = frame.copy()

    # Draw box
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score > 0.8:
            print(box)
            b = box.astype(int)
            color = label_color(label)
            draw_box(draw, b, color=color)
            caption = "{} {:.3f}".format(labels_to_names[label], score)
            draw_caption(draw, b, caption)

    # show the frame
    cv2.imshow("Test out", draw)

    key = cv2.waitKey(1)

    # quit camera if 'q' key is pressed
    if key & 0xFF == ord("q"):
        break

camera.release()
cv2.destroyAllWindows()

[ 761.2831    82.0421  1056.4431   475.00098]
[ 772.08545   81.22692 1068.1481   466.00607]
[ 759.82635  109.76564 1049.6697   501.69666]
[ 762.2282   118.87513 1053.1583   511.70966]
[ 774.3428   126.60433 1077.6116   514.9403 ]
[ 766.52795  125.61697 1071.8877   522.0162 ]
[ 769.9345   127.52517 1071.391    525.4625 ]
[ 783.2609   131.99385 1063.3044   511.46408]
[ 771.7144  128.4459 1056.4603  497.0247]
[ 764.8234   137.19223 1038.2501   487.36356]
[178.9344  310.79855 473.35184 674.3293 ]
[ 775.7315   304.6386  1025.6185   597.16907]
[ 735.3328   106.83909 1023.0659   506.67496]
[ 729.2299     63.888924 1066.3662    523.351   ]
[ 736.98285   76.32351 1068.8201   485.9489 ]
[ 720.65826   78.40246 1058.0881   489.45468]
[ 719.2726    83.80886 1050.0455   467.54376]
[ 713.74976   81.57744 1042.2067   470.8964 ]
[ 739.5746    72.33621 1076.1863   476.35568]
[ 736.20593   81.87418 1065.2808   468.36926]
[494.11713 321.2045  830.39453 689.8051 ]
[211.72346 294.9925  463.30182 582.5089 ]
